# Reading Geographical Coordinates from CSV file, converting it to dataframe and making required dataframe by merging it with combined Neighbourhood dataframe 

### Importing necessary libraries

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as urlreq
import lxml

### Reading Wikipedia page for scraping

In [4]:
with urlreq.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") as response:
    PCC_html = response.read()

In [5]:
#print(PCC_html)

### parsing the wiki webpage html using beautiful soup package, and displayed

In [6]:
soup = BeautifulSoup(PCC_html, 'html.parser')
#print(soup.prettify())

### Extracting the table from html using beautiful soup object

In [7]:
table = soup.find_all('table')[0] # finding the first table in the html
#print(str(table))

### Conversion of html table to string and then conversion to dataframes using pandas function read_html()

In [8]:
df = pd.read_html(str(table))[0] # conversion of html to string and then conversion to dataframes
#df.head(10)

### Ignoring cells with a borough that is Not assigned

In [9]:
df_with_borough = df[df['Borough'] != "Not assigned"]
#df_with_borough.head()

### Resetting the index

In [10]:
df_with_borough.reset_index(inplace=True, drop=True)
#df_with_borough.head(7)

### Here we can see the Borough which have Neighbourhood not assigned so it is required to be coppied with the Borough name

In [11]:
df_same_borough_nbr = df_with_borough

#  Ignore cells with a borough that is Not assigned
p = df_same_borough_nbr.loc[df_same_borough_nbr['Neighbourhood'] == "Not assigned"]
#p.head(7)

### Copying Neighbourhood with Borough that has "Not assigned"

#### It can be seen that the Borough Queen's Park is coppied in the Neighbourhood as well

In [12]:
df_same_borough_nbr.iloc[p.index[0]]['Neighbourhood'] = df_same_borough_nbr.iloc[p.index[0]]['Borough']
#df_same_borough_nbr.head(10)

### function describe() tells the total number of Postcode, Borough and Neighbourhood repeatition with unique number of Postcodes, here it can be seen the difference between the repeatition and unique Postcode where both should be same

In [13]:
df_same_borough_nbr.describe()

,Postcode,Borough,Neighbourhood
count,211,211,211
unique,103,11,209
top,M9V,Etobicoke,St. James Town
freq,8,45,2


### Grouping the table by Postcode and accessing a specific group can be done as follows

In [14]:
pp=df_same_borough_nbr.groupby('Postcode')
k=pp.get_group('M5A')
#k

### Creating a new data frame with the same column names for combinded Neighbourhood

In [15]:
newDF = pd.DataFrame(columns = ['Postcode', 'Borough', 'Neighbourhood'])
#newDF

### Unique Postcodes are determined using pandas unique() function for accessing each group of Postcode seperately, 
### Single Neighbourhood of a Borough is copied as it is but Multiple Neighbourhood of a Borough is concatenated using join() function seperated by comma
### apply() function is used with axis = 0 for accessing each row of Neighbourhood which has join() function for concatination as described above
### Lastly, a row of dataframe is created and appended based on Postcode, Borough and respective Combined Neighbourhood in the new dataframe

In [17]:

# getting unique post codes for iteration to all the groups based on 'postcode' one by one for extracting neighbourhood
df_uniquePostcodes = df_same_borough_nbr['Postcode'].unique()

for PC in df_uniquePostcodes:
    Total_Nbr_of_PC = pp.get_group(PC)
    
    # copying the rows with no multiple Neighbourhoods
    if len(Total_Nbr_of_PC)==1:
        POSTCODE = Total_Nbr_of_PC.iloc[0,0]
        BOROUGH = Total_Nbr_of_PC.iloc[0,1]
        NEIGHBOURHOOD = Total_Nbr_of_PC.iloc[0,2]
        newDF = newDF.append(pd.DataFrame({'Postcode':POSTCODE,'Borough':BOROUGH, 'Neighbourhood':NEIGHBOURHOOD}, index=[0]) , ignore_index=True)
    elif len(Total_Nbr_of_PC)>1:
        # Following line combines all the Neighbourhoods of a group in rows seperated with comma
        tt = Total_Nbr_of_PC[['Neighbourhood']].apply(lambda alpha: ','.join(alpha),axis=0)
        POSTCODE = Total_Nbr_of_PC.iloc[0,0]
        BOROUGH = Total_Nbr_of_PC.iloc[0,1]
        NEIGHBOURHOOD = tt[0]
        
        #print(Total_Nbr_of_PC,', Length = ', len(Total_Nbr_of_PC))
        #print('Postcode = ', Total_Nbr_of_PC.iloc[0,0])
        #print('Borough = ', Total_Nbr_of_PC.iloc[0,1])
        #print('Appended Nbr = ', tt[0], '\n')
        
        newDF = newDF.append(pd.DataFrame({'Postcode':POSTCODE,'Borough':BOROUGH, 'Neighbourhood':NEIGHBOURHOOD}, index=[0]) , ignore_index=True)

    #print(pp.get_group(PC))
    #print(PC)
    
newDF.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


### Checking the size of table shows that now the table have rows equal to unique Postcodes

In [18]:
newDF.shape

(103, 3)

### Also describe() function shows that now the total count of rows of table is equal to the unique Postcodes

In [19]:
newDF.describe()

,Postcode,Borough,Neighbourhood
count,103,103,103
unique,103,11,103
top,M2R,North York,Berczy Park
freq,1,24,1


# Making Data Frame with Latitude and Longitude

## Fetching Geolocation Data into csv file

In [20]:
!wget -q -O 'Geospatial_DATA.csv' 'http://cocl.us/Geospatial_data'

## Reading the csv file and converting into dataframe using pandas function read_csv()

In [23]:
GeoData = pd.read_csv('Geospatial_DATA.csv')
GeoData.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


## The column name Postal Code in GeoData dataframe is not equal to the column name Postcode in NewDF dataframe so it is renamed for merging

In [24]:
GeoData.rename(columns={'Postal Code':'Postcode'}, inplace=True)
GeoData.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging the two dataframes newDF and GeoData to get required dataframe of Postcode with its respective latitude and longitude coordinates

In [26]:
df_PC_with_LatLong = pd.merge(newDF, GeoData, on='Postcode', how='inner')
df_PC_with_LatLong.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
